In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
prices = pd.read_csv('sample_data/3assets_prices.csv', parse_dates=['Date']).set_index('Date')
rets = pd.read_csv('sample_data/3assets_returns.csv', parse_dates=['Date']).set_index('Date')
rets.columns=['Asset_A','Asset_B','Asset_C']
CL=0.99; z=2.3263478740408408; total_value=1_000_000.0; weights=np.array([0.5,0.3,0.2])
def port(r,w): return (r*w).sum(axis=1)
def vhist(x,cl): return -np.quantile(x,1-cl)
def ehist(x,cl): q=np.quantile(x,1-cl); t=x[x<=q]; return -t.mean()
def vnorm(m,s,cl): return -(m - z*s)
def enorm(m,s,cl): phi=(1/np.sqrt(2*np.pi))*np.exp(-0.5*z*z); return -(m - s*(phi/(1-cl)))
p = port(rets, weights)
res={}
for name in ['Asset_A','Asset_B','Asset_C','Portfolio']:
    x = p if name=='Portfolio' else rets[name]
    res[name]={'Hist_VaR_1d':vhist(x,CL),'Hist_ES_1d':ehist(x,CL),'Norm_VaR_1d':vnorm(x.mean(),x.std(ddof=1),CL),'Norm_ES_1d':enorm(x.mean(),x.std(ddof=1),CL)}
pd.DataFrame(res).T

,Hist_VaR_1d,Hist_ES_1d,Norm_VaR_1d,Norm_ES_1d
Asset_A,0.024065,0.028188,0.025838,0.029856
Asset_B,0.015738,0.018232,0.016859,0.019365
Asset_C,0.043617,0.048245,0.043277,0.049872
Portfolio,0.017313,0.020358,0.018810,0.021750


In [ ]:
mu = rets.mean().values; cov = rets.cov().values
sims = np.random.multivariate_normal(mu, cov, size=40000)
ps = sims @ weights
mc_var = -np.quantile(ps, 1-CL); mc_es = -ps[ps<=np.quantile(ps,1-CL)].mean()
mc_var, mc_es

(np.float64(0.018781246635566906), np.float64(0.021973652877018843))

In [ ]:
last_prices = prices.iloc[-1].values; units = total_value*weights/last_prices
values = units*last_prices; half_spread=np.array([0.0020,0.0015,0.0030]); ttl=3; impact=np.array([0.0010,0.0008,0.0015])
base_var_1d = -(p.mean() - z*p.std(ddof=1))*total_value
add_on = (values*half_spread*np.sqrt(ttl)).sum() + (values*impact).sum()
LVaR_1d = base_var_1d + add_on
base_var_1d, add_on, LVaR_1d

(np.float64(18809.614582457456),
 np.float64(4590.704155516198),
 np.float64(23400.318737973656))